In [1]:
# Load libraries 
import pandas as pd
from PIL import Image
import os
from pillow_heif import register_heif_opener
import cv2
import pytesseract
import re
from pyzbar.pyzbar import decode
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe" #definig the path to where tesseract is located

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reading the data 
product_data = pd.read_excel('Product data.xlsx')
product_data

,Barcode,Unnamed: 1,Ingredient List,Name
0,3600542399326,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...
1,8606029266766,8606029266766,Silicone,Velnea Fingering Silik.Lila
2,4049639429550,4049639429550,"aqua (water),stearic acid,copernicia cerifera ...","Magic Finish 5-In-1 Hybrid Mascara\nVolume, Le..."
3,4011700740291,4011700740291,"butane,alcohol,propane,parfum (fragrance),dipr...",Echt Kï¿½Lnisch Wasser Aerosol Deodorant Spray...
4,4011700740475,4011700740475,"sodium palmate,sodium palm kernelate,aqua (wat...",Echt Kï¿½Lnisch Wasser Cream Soap\nMildly Clea...
...,...,...,...,...
13669,8034063521822,8034063521822,"Aqua [Water], Sodium coco-sulfate, Cocamidopro...",Shampoo Leaves Strengthening Anti-Age With Gin...
13670,6001051004959,6001051004959,"Aqua, Alcohol Denat., Butyl Methoxydibenzoylme...",Sun Babies & Kids Kids' Sun Spray Spf 50+
13671,5060447940494,5060447940494,"Aqua (Water), Octocrylene, Alcohol Denat., Gly...",Sun Cream Spf 30
13672,4005900870407,4005900870407,"ALPHA-ISOMETHYL IONONE, *******, BUTYL METHOXY...",Sun Oil Spray Tropical Bronze Spf 6


In [4]:
product_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13674 entries, 0 to 13673
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Barcode          13674 non-null  int64 
 1   Unnamed: 1       13674 non-null  int64 
 2   Ingredient List  13674 non-null  object
 3   Name             13674 non-null  object
dtypes: int64(2), object(2)
memory usage: 427.4+ KB


In [5]:
# Checking to see if there are any duplicates
duplicates = product_data[product_data.duplicated('Barcode')]
duplicates

,Barcode,Unnamed: 1,Ingredient List,Name


In [6]:
# Dropping unnecessary columns
product_data = product_data.drop('Unnamed: 1', axis=1, inplace=False)

In [7]:
# Chcking to see if there are null in the 'Ingredient List' column
none_values= product_data[product_data['Ingredient List'].isna()]
none_values

,Barcode,Ingredient List,Name


In [8]:
# This code can be used for cleaning the Ingredient List column
# def clean_and_capitalize(text):
#     if isinstance(text, str):
#     # Remove special characters using regex
#         cleaned_text = re.sub(r'[^a-zA-Z0-9*,\s\[\]()]-', '', text)
    
#     # Capitalize the first letter of each word
#         cleaned_text = cleaned_text.title()
    
#         return cleaned_text
#     else:
#         return str(text)

# # Apply the cleaning function to the 'Ingredient List' column
# product_data['Ingredient List'] = product_data['Ingredient List'].apply(clean_and_capitalize)

product_data

,Barcode,Ingredient List,Name
0,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...
1,8606029266766,Silicone,Velnea Fingering Silik.Lila
2,4049639429550,"aqua (water),stearic acid,copernicia cerifera ...","Magic Finish 5-In-1 Hybrid Mascara\nVolume, Le..."
3,4011700740291,"butane,alcohol,propane,parfum (fragrance),dipr...",Echt Kï¿½Lnisch Wasser Aerosol Deodorant Spray...
4,4011700740475,"sodium palmate,sodium palm kernelate,aqua (wat...",Echt Kï¿½Lnisch Wasser Cream Soap\nMildly Clea...
...,...,...,...
13669,8034063521822,"Aqua [Water], Sodium coco-sulfate, Cocamidopro...",Shampoo Leaves Strengthening Anti-Age With Gin...
13670,6001051004959,"Aqua, Alcohol Denat., Butyl Methoxydibenzoylme...",Sun Babies & Kids Kids' Sun Spray Spf 50+
13671,5060447940494,"Aqua (Water), Octocrylene, Alcohol Denat., Gly...",Sun Cream Spf 30
13672,4005900870407,"ALPHA-ISOMETHYL IONONE, *******, BUTYL METHOXY...",Sun Oil Spray Tropical Bronze Spf 6


In [9]:
# Converting heic to jpg

# Get list of HEIF and HEIC files in directory
directory = r'C:\Users\user\Data Science\Final project\Images data'
register_heif_opener()

files = [f for f in os.listdir(directory) if f.endswith('.heic') or f.endswith('.heif')]

# Convert each file to JPEG and save in the same directory
for filename in files:
    heic_path = os.path.join(directory, filename)
    output_path = os.path.splitext(heic_path)[0] + '.jpg'

    with Image.open(heic_path) as image:
        image.convert('RGB').save(output_path)

In [10]:
# def image_reader(image_path):
    
#     image = cv2.imread(image_path)
    
#     #Preprocessing the image
#     gray= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blur = cv2.GaussianBlur(gray,(5,5),0)
#     cv2.imwrite(image_path +"blur.jpg", blur)

#     thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
#     cv2.imwrite(image_path +"thresh.jpg", thresh)

#     th2 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
#             cv2.THRESH_BINARY,11,2)
#     cv2.imwrite(image_path +"th2.jpg", th2)

#     th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#                                 cv2.THRESH_BINARY,11,2)
#     cv2.imwrite(image_path +"th3.jpg", th3)

#     def noise_removal(image):
#         import numpy as np
#         kernel = np.ones((1, 1), np.uint8)
#         image = cv2.dilate(image, kernel, iterations=1)
#         kernel = np.ones((1, 1), np.uint8)
#         image = cv2.erode(image, kernel, iterations=1)
#         image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
#         image = cv2.medianBlur(image, 3)
#         return (image)
    
#     image_1 = noise_removal(th2)
#     cv2.imwrite(image_path +"noiseth2.jpg", image_1)
    
#     image_2= noise_removal(th3)
#     cv2.imwrite(image_path +"noiseth3.jpg", image_2)
#     #Extracting the text from the image 
#     result =pytesseract.image_to_string(thresh, lang='eng', config=r'--oem 3 --psm 6')
#     result= result.replace('\n','')

#     #Finding the Ingredinets list
#     ing_pattern = re.compile(r'INGREDIENTS:\s*([\w\s,]+)', re.IGNORECASE) 
#     ing_matches = re.findall(ing_pattern, result)

#     if ing_matches:
#         ingredients_list = ing_matches[0].split(',')
#         ingredients_str = ', '.join(ingredients_list)
#         print(ingredients_str)
#         return ingredients_str
#     print('Cannot read the Ingredients list. Try with Barcode')

# The code above is us trying different technics to imporve the reading of the images
def image_reader(image_path):
    image = cv2.imread(image_path)
    
    # Preprocessing the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    def noise_removal(image):
        import numpy as np
        kernel = np.ones((1, 1), np.uint8)
        image = cv2.dilate(image, kernel, iterations=1)
        kernel = np.ones((1, 1), np.uint8)
        image = cv2.erode(image, kernel, iterations=1)
        image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
        image = cv2.medianBlur(image, 3)
        return image
    
    image = noise_removal(thresh)
    
    # Extracting the text from the image
    result = pytesseract.image_to_string(image, lang='eng')
    result = result.replace('\n', '')

    # Finding the Ingredients list
    ing_pattern = re.compile(r'INGREDIENTS:\s*([\w\s,]+)', re.IGNORECASE) # finding the word 'INGREDIENTS' no matter how is written
    ing_matches = re.findall(ing_pattern, result)

    if ing_matches:
        ingredients_list = ing_matches[0].split(',')
        ingredients_string = ','.join(ingredients_list)

        return print(ingredients_string)
    else:
        print('Cannot read the Ingredients list. Try with Barcode') 

In [11]:
image = image_reader(r"C:\Users\user\Data Science\Final project\Images data\IMG_6949.jpg")

Sodium Palmate, Aqua,Glycerin, Sodium Palm Kernelate, Parfum,Palm Kernel Acid, Rubus Idaeus Fruit Extract,Vaccinium Macrocarpon Fruit Extract, PrunusAmygdalus Dulcis Seed Extract, Zea MaysStarch, Xanthan Gum, Citric Acid, SodiumBenzoate, Potassium Sorbate, TetrasodiumGlutamate Diacetate, Sodium Citrate, SodiumChloride, Benzyl Salicylate, Geraniol, BenzylAlcohol, Hexyl Cinnamal, Linalool, Cl 77891


In [12]:
# Reading barcode
  
# Make one method to decode the barcode 
def barcode_reader(image_path):
    
    img = cv2.imread(image_path)
    detectedBarcodes = decode(img)
      
    if not detectedBarcodes:
        pass
    else:
       
        for barcode in detectedBarcodes:  
            (x, y, w, h) = barcode.rect # locating the barcode position 
            cv2.rectangle(img, (x-10, y-10),
                          (x + w+10, y + h+10), 
                          (255, 0, 0), 2)
             
            if barcode.data!=b"":
                barcode = int(barcode.data.decode('utf-8'))
                print(barcode)


In [13]:
barcode_reader(r"C:\Users\user\Data Science\Final project\Images data\IMG_6949.jpg")

4045612009474


In [14]:
# Function to read ingrediends and barcodes 
def processing_image(image_path):
    global product_data
    # using ImageReader to read ingredient list
    ingredient_list = image_reader

    # using BarcodeReader to read barcode
    barcode = barcode_reader

    if ingredient_list() is not None or barcode() is not None:
        if barcode is not None and barcode not in product_data['Barcode'].values: # if both ingredients list and barcode are detected
            new_row = {'Barcode': barcode, 'Ingredient List': ingredient_list, 'Name': 'N/A'}
            product_data = product_data.append(new_row, ignore_index=True)
            return product_data
        elif barcode is not None: # if only barcode is detected
            new_row = {'Barcode': barcode, 'Ingredient List': 'Not readable', 'Name': 'N/A'}
            product_data = product_data.append(new_row, ignore_index=True)
            return product_data
        else: # if only ingredients list is detected
            new_row = {'Barcode': 'Not in the picture', 'Ingredient List': ingredient_list, 'Name': 'N/A'}
            product_data = product_data.append(new_row, ignore_index=True)
            return product_data
            
    else:
        return 'Take a picture again.'

    return product_data

In [15]:
# Final functions
def image_reader(image_path):
    image = cv2.imread(image_path)
    
    # Preprocessing the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    def noise_removal(image):
        import numpy as np
        kernel = np.ones((1, 1), np.uint8)
        image = cv2.dilate(image, kernel, iterations=1)
        kernel = np.ones((1, 1), np.uint8)
        image = cv2.erode(image, kernel, iterations=1)
        image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
        image = cv2.medianBlur(image, 3)
        return image
    
    image = noise_removal(thresh)
    
    # Extracting the text from the image
    result = pytesseract.image_to_string(image, lang='eng')
    result = result.replace('\n', '')

    # Finding the Ingredients list
    ing_pattern = re.compile(r'INGREDIENTS:\s*([\w\s,]+)', re.IGNORECASE)
    ing_matches = re.findall(ing_pattern, result)

    if ing_matches:
        ingredients_list = ing_matches[0].split(',')
        ingredients_string = ','.join(ingredients_list)

        return ingredients_string
    else:
        return None

def barcode_reader(image_path):
    img = cv2.imread(image_path)
    detectedBarcodes = decode(img)
      
    if not detectedBarcodes:
        return None
    else:
        for barcode in detectedBarcodes:  
            (x, y, w, h) = barcode.rect
            cv2.rectangle(img, (x-10, y-10),
                          (x + w+10, y + h+10), 
                          (255, 0, 0), 2)
             
            if barcode.data != b"":
                barcode_value = barcode.data.decode('utf-8')
                return barcode_value
    return None

def processing_image(image_path):
    global product_data
    
    # using ImageReader to read ingredient list
    ingredient_list = image_reader(image_path)

    # using BarcodeReader to read barcode
    barcode = barcode_reader(image_path)

    if ingredient_list is not None or barcode is not None:
        if barcode is not None and barcode not in product_data['Barcode'].values:
            new_row = {'Barcode': barcode, 'Ingredient List': ingredient_list, 'Name': 'N/A'}
            product_data = product_data.append(new_row, ignore_index=True)
            return product_data
        elif barcode is not None:
            new_row = {'Barcode': barcode, 'Ingredient List': 'Not readable', 'Name': 'N/A'}
            product_data = product_data.append(new_row, ignore_index=True)
            return product_data
        else:
            new_row = {'Barcode': 'Not in the picture', 'Ingredient List': ingredient_list, 'Name': 'N/A'}
            product_data = product_data.append(new_row, ignore_index=True)
            return product_data
    else:
        return 'Take a picture again.'

    return product_data

In [18]:
image_folder = r'C:\Users\user\Data Science\Final project\Images data'

# Iterate through all files in the specified folder
for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')): 
        image_path = os.path.join(image_folder, filename)
        result = processing_image(image_path)


In [19]:
# Viewing the final result
#pd.set_option('display.max_rows', None)
product_data.tail(300)

,Barcode,Ingredient List,Name
13524,8809416470764,"135Ml Propolis Extract, Butylene Glycol, Glyce...",One Step Moisture Up Pad
13525,8809612852524,"80Ml- Water / Aqua / Eau, Butylene Glycol, Pro...",Green Tea Sleeping Mask
13526,8809803522083,"70Ml- Water, Butylene Glycol, Glycerine, Treha...",Water Sleeping Mask Ex (Cherry Blossom)
13527,8809598453500,"70 Pads- Propolis Extract, Butylene Glycol, Be...",Full Fit Propolis Synergy Pad
13528,8809416470733,"50G Propolis Extract, Butylene Glycol, Glyceri...",Ultimate Moisturizing Honey Overnight Mask
...,...,...,...
13819,4084200771904,AQUA,N/A
13820,Not in the picture,"Aqua, Palmitic Acid, Triethanolamine, Oleth",N/A
13821,4056489431091,None,N/A
13822,0738678247778,None,N/A
